## Header 

In [ ]:
# Code to set up the assignment
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/
#!mkdir -p 10714
%cd /content/drive/MyDrive/10714
#!git clone https://github.com/dlsys10714/hw4.git
%cd /content/drive/MyDrive/10714/hw4/

!pip3 install --upgrade --no-deps git+https://github.com/dlsys10714/mugrade.git
!pip3 install pybind11

In [ ]:
!make

-- Found pybind11: /usr/local/lib/python3.8/dist-packages/pybind11/include (found version "2.10.3")
-- Found cuda, building cuda backend
NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.

-- Configuring done
-- Generating done
-- Build files have been written to: /content/drive/MyDrive/10714/hw4_/build
make[1]: Entering directory '/content/drive/MyDrive/10714/hw4_/build'
make[2]: Entering directory '/content/drive/MyDrive/10714/hw4_/build'
make[3]: Entering directory '/content/drive/MyDrive/10714/hw4_/build'
Consolidate compiler generated dependencies of target ndarray_backend_cpu
make[3]: Leaving directory '/content/drive/MyDrive/10714/hw4_/build'
[  0%] Built target ndarray_backend_cpu
make[3]: Entering directory '/content/drive/MyDrive/10714/hw4_/build'
make[3]: Leaving directory '/content/drive/MyDrive/10714/hw4_/build'
[ 50%] Built target ndarray_backend_cuda
make[2]: Leaving directory '/

In [ ]:
import sys
sys.path.append('./python')

## Transformers Implementation 


### Single Head

#### Numpy and Torch 

In [ ]:
import numpy as np 
import torch.nn as nn 
import torch 

In [ ]:
def softmax(z):
    z= np.exp(z-z.max(axis=-1,keepdims=True))
    z= z/z.sum(axis=-1,keepdims=True) 
    return z

In [ ]:
def self_attntion(x,mask,W_QKV,W_out): 
    K,Q,V= np.split(x@W_QKV,3,axis= -1)
    attn = softmax(K@Q.swapaxes(-1,-2)/np.sqrt(x.shape[-1]) +mask)
    return attn@V@W_out,attn

In [ ]:
T,d = 100,64 
pytorch_attn = nn.MultiheadAttention(d,1,bias=False,batch_first=True) 
M=torch.triu(-float("inf")*torch.ones(T,T),1)
X = torch.randn(1,T,d)
Y_,A_ = pytorch_attn(X,X,X,attn_mask=M)

In [ ]:
pytorch_attn.in_proj_weight

Parameter containing:
tensor([[-0.0639, -0.1004,  0.1387,  ...,  0.0807, -0.0256,  0.0107],
        [-0.1129, -0.0043,  0.0175,  ...,  0.0309,  0.1409, -0.0381],
        [ 0.0396, -0.0262, -0.1473,  ..., -0.0444, -0.0025, -0.1250],
        ...,
        [-0.0588,  0.1344,  0.0573,  ..., -0.0036,  0.1160, -0.0110],
        [ 0.0961,  0.1281,  0.0603,  ...,  0.1058, -0.0673,  0.1057],
        [-0.1393,  0.0834,  0.0303,  ..., -0.0488, -0.0271,  0.0366]],
       requires_grad=True)

In [ ]:
pytorch_attn.in_proj_weight.shape

torch.Size([192, 64])

In [ ]:
y,a = self_attntion(X[0].numpy(),
              M.detach().numpy(),
              pytorch_attn.in_proj_weight.detach().numpy().T,
              pytorch_attn.out_proj.weight.detach().numpy().T)

In [ ]:
np.linalg.norm(y-Y_.detach().numpy())

2.1186613e-06

#### needle 

In [ ]:
import needle as ndl
from needle import backend_ndarray as nd

In [ ]:
x = np.random.rand(4,5,3)
y = np.random.rand(4,3,7)
z = x@y 
z.shape

(4, 5, 7)

In [ ]:
A = ndl.Tensor(nd.array(x), device=ndl.cpu())


In [ ]:
type(A)

needle.autograd.Tensor

In [ ]:
list_= []
list_+= [A]
list_+= [A]
list_+= [A]
ndl.stack(list_,axis= 0)

needle.Tensor([[[[0.83470535 0.2566686  0.6160921 ]
   [0.84934247 0.23257907 0.3750166 ]
   [0.69505304 0.36830238 0.00763387]
   [0.04668795 0.64148545 0.41045737]
   [0.45124054 0.23824914 0.04564811]]

  [[0.01622594 0.18151471 0.2542918 ]
   [0.09291264 0.8217102  0.15739997]
   [0.18919791 0.57033896 0.21660209]
   [0.7835239  0.93566793 0.28094074]
   [0.41302624 0.25173417 0.07278667]]

  [[0.27279794 0.3372212  0.07701581]
   [0.02709366 0.16226222 0.65644914]
   [0.20317753 0.80822873 0.07684939]
   [0.28478497 0.40078703 0.18103081]
   [0.23169583 0.9919206  0.42142302]]

  [[0.33048588 0.25607657 0.47141388]
   [0.91839296 0.2121399  0.41372746]
   [0.80201006 0.4352404  0.58073837]
   [0.8987972  0.8565136  0.97653604]
   [0.9503554  0.27668574 0.28438362]]]


 [[[0.83470535 0.2566686  0.6160921 ]
   [0.84934247 0.23257907 0.3750166 ]
   [0.69505304 0.36830238 0.00763387]
   [0.04668795 0.64148545 0.41045737]
   [0.45124054 0.23824914 0.04564811]]

  [[0.01622594 0.1815147

In [ ]:
!python dummy_test.py 

after numpy the shape is (4, 5, 7)
A <class 'needle.autograd.Tensor'>
<class 'needle.backend_ndarray.ndarray.NDArray'>
<class 'needle.backend_ndarray.ndarray.NDArray'>
<class 'needle.backend_ndarray.ndarray.NDArray'>
<class 'needle.backend_ndarray.ndarray.NDArray'>
c shape is (5, 7),type <class 'needle.backend_ndarray.ndarray.NDArray'>
c shape is (5, 7),type <class 'needle.backend_ndarray.ndarray.NDArray'>
c shape is (5, 7),type <class 'needle.backend_ndarray.ndarray.NDArray'>
c shape is (5, 7),type <class 'needle.backend_ndarray.ndarray.NDArray'>
length of output 4
after numpy the shape is (4, 5, 7)
3.648436804215749e-07


In [ ]:
!python -m pytest -l -v -k "bbm_forward"

============================= test session starts ==============================
platform linux -- Python 3.8.16, pytest-3.6.4, py-1.11.0, pluggy-0.7.1 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/drive/MyDrive/10714/hw4_, inifile:
plugins: typeguard-2.7.1
collected 1805 items / 1803 deselected                                         

tests/test_ext_ops.py::test_bbm_forward[params0-device0] PASSED          [ 50%]
tests/test_ext_ops.py::test_bbm_forward[params0-device1] PASSED          [100%]

================== 2 passed, 1803 deselected in 2.49 seconds ===================


In [ ]:
A = ndl.Tensor(nd.array(x), device=ndl.cpu())
B = ndl.Tensor(nd.array(y), device=ndl.cpu())
z2 = ndl.bmm(A,B)
z2.shape

AssertionError: ignored

In [ ]:
x[1,:,:].shape

(5, 3)

In [ ]:
softmax(K@Q.swapaxes(-1,-2)/np.sqrt(x.shape[-1]) +mask)

In [ ]:
Q.shape

torch.Size([100, 64])

In [ ]:
Z1= K@Q.swapaxes(-1,-2)
Z2 = ndl.Tensor(nd.array(K),device = ndl.cpu()) @ ndl.Tensor(nd.array(Q),device = ndl.cpu()).transpose((-1,-2))
Z1= softmax(Z1.numpy())
#Z2 = ndl.logsumexp(Z2,axes = -1)
np.linalg.norm(Z2.numpy() - Z1)


389.3541

In [ ]:
assert 1==2 , "ass"

AssertionError: ignored

In [ ]:
def self_attntion_needle(x,mask,K,Q,V,W_out): 
    #K,Q,V= ndl.split(x@W_QKV,axis= -1)
    interm = K@Q.transpose((-1,-2))
    attn = ndl.exp(ndl.logsumexp(interm/np.sqrt(x.shape[-1]) +mask,axes=-1))
    return attn@V@W_out,attn

In [ ]:
K,Q,V = np.split(X[0]@pytorch_attn.in_proj_weight.detach().numpy().T,3,-1)

In [ ]:
y1,a1= self_attntion_needle(ndl.Tensor(nd.array(X[0]), device=ndl.cpu()),
                          ndl.Tensor(nd.array(M.detach()), device=ndl.cpu()),
                          ndl.Tensor(nd.array(K), device=ndl.cpu()),
                          ndl.Tensor(nd.array(Q), device=ndl.cpu()),
                          ndl.Tensor(nd.array(V), device=ndl.cpu()),
                          ndl.Tensor(nd.array(pytorch_attn.out_proj.weight.detach().numpy().T), device=ndl.cpu()))

inter sshape (100, 100) , mask (100, 100), x 
am here 3 


In [ ]:
np.linalg.norm(y1.numpy()-Y_.detach().numpy())

19494.152

In [ ]:
Y_.detach().numpy()

array([[[ 1.4319317 ,  1.1891581 , -0.6933644 , ...,  0.65494126,
          0.40657172,  0.52924234],
        [ 0.7774052 ,  0.5540716 , -0.4839186 , ...,  0.43025815,
          0.04043669,  0.47467348],
        [ 0.6789432 ,  0.5909815 , -0.27973345, ...,  0.29926848,
         -0.13691753,  0.467305  ],
        ...,
        [ 0.006937  , -0.01924795,  0.01213193, ..., -0.03500503,
          0.06926697, -0.01781674],
        [ 0.09237896,  0.08156796, -0.03369059, ..., -0.00229773,
          0.08049973, -0.04902685],
        [ 0.06407653, -0.00245799,  0.06508794, ..., -0.02389152,
          0.0328558 ,  0.00353461]]], dtype=float32)

In [ ]:
y1.numpy() 

array([[ 4.02142000e+00, -2.31199121e+00,  2.98760056e+00, ...,
        -1.40650272e+00,  9.30363560e+00, -1.43880665e+00],
       [ 1.82752724e+01, -1.16519232e+01,  1.37074995e+01, ...,
        -6.65031195e+00,  4.59798355e+01, -8.53326416e+00],
       [ 2.05431976e+01, -1.69026840e+00,  5.93793869e+00, ...,
        -7.12287605e-01,  3.52013893e+01, -2.90395403e+00],
       ...,
       [ 3.91363647e+02, -1.41149582e+02,  3.09109863e+02, ...,
        -2.29007301e+01,  6.58865295e+02, -1.57089462e+01],
       [ 5.21057068e+02, -1.57911133e+02,  1.93444427e+02, ...,
        -1.15784973e+02,  9.66197388e+02, -1.45222260e+02],
       [ 2.85343384e+02, -2.74345541e+00, -7.62693939e+01, ...,
        -8.90989151e+01,  5.12652954e+02, -6.94488754e+01]], dtype=float32)

## Testing Split

In [ ]:
!python dummy_test.py  

am here 0
am here 1 
am here 2 
arr shape here is (15, 108) an new sshape (15, 4, 9, 3), fially axes [2, 0, 1, 3]
5.361648239661069e-07


In [ ]:
import numpy as np

In [ ]:
x= np.random.rand(5,7,8,9)


In [ ]:
import needle.nn as nn 

In [ ]:
x= np.random.rand(10,5,4,9)
y= softmax(x)
print(y.shape)

(10, 5, 4, 9)


In [ ]:
y2 = nn.Softmax()(ndl.Tensor(nd.array(x), device=ndl.cpu()))

AttributeError: ignored

In [ ]:
y= ndl.Tensor(nd.array(x), device=ndl.cpu())

In [ ]:
y.shape

(5, 7, 8, 9)

In [ ]:
l2= ndl.logsumexp(y,axes= 3)
l2 = l2.reshape((5, 7, 8, 1))
l2= l2.broadcast_to((5, 7, 8, 9))
softmax_ = ndl.exp(y - l2)


In [ ]:
z1 = softmax(y.numpy())

In [ ]:
np.linalg.norm(softmax_.numpy() - z1)

7.349498e-07

In [ ]:
import scipy

In [ ]:
l1 = logsumexp(y.numpy(), axis=-1, keepdims=True)
z2 = np.exp(y.numpy() - l1)

In [ ]:
from scipy.special import logsumexp


In [ ]:
y.shape

(5, 7, 8, 9)

In [ ]:
l1 = logsumexp(y.numpy(), axis=3, keepdims=True)
l2 = ndl.logsumexp(y,axes =  3)

In [ ]:
print(l2.shape)
print(l1.shape)

(5, 7, 8)
(5, 7, 8, 1)


In [ ]:
l2 = l2.reshape((5, 7, 8, 1))


In [ ]:
np.linalg.norm(l2.numpy()-l1)

1.7357135e-06

In [ ]:
torch.logsumexp(A_t, dim=t_axes).numpy()

============================= test session starts ==============================
platform linux -- Python 3.8.16, pytest-3.6.4, py-1.11.0, pluggy-0.7.1 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/drive/MyDrive/10714/hw4_, inifile:
plugins: typeguard-2.7.1
collected 1805 items / 1687 deselected                                         

tests/test_nd_backend.py::test_ewise_fn[cpu-shape0-divide] PASSED        [  0%]
tests/test_nd_backend.py::test_ewise_fn[cpu-shape0-subtract] PASSED      [  1%]
tests/test_nd_backend.py::test_ewise_fn[cpu-shape1-divide] PASSED        [  2%]
tests/test_nd_backend.py::test_ewise_fn[cpu-shape1-subtract] PASSED      [  3%]
tests/test_nd_backend.py::test_ewise_fn[cuda-shape0-divide] FAILED       [  4%]
tests/test_nd_backend.py::test_ewise_fn[cuda-shape0-subtract] FAILED     [  5%]
tests/test_nd_backend.py::test_ewise_fn[cuda-shape1-divide] FAILED       [  5%]
tests/test_nd_backend.py::test_ewise_fn[cuda-shape1-subtract] FAILED     [  6%]
tests